# Ham or Spam
Actual message or a spam

In [1]:
import pandas as pd
from warnings import filterwarnings
# filterwarnings('ignore')

In [1]:
df = pd.read_csv('restaurant.csv', encoding='latin-1')
df.head()

NameError: name 'pd' is not defined

In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.columns = ['class', 'text']
df.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


We will predict if class is ham or not. This is a classification problem.

In [5]:
import seaborn as sns

In [6]:
sns.countplot(x=df['class'], palette='bright');

In [7]:
df.shape

(5572, 2)

In [8]:
df['class'].value_counts()

class
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
df['class'].isnull().sum()

0

In [10]:
df['class'] = df['class'].map({ 'ham': 0, 'spam': 1 })
df.head()

,class,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Starting an NLP project

1. Convert all to lowecase
2. Remove punctiation
3. Remove numbers
4. Remove newline characters
5. Remove stopwords (unnecessary words)
6. Tokenize
7. use Lemma & Stemma to get the stem of the words
8. Vectorize

In [12]:
def nlp_prep(col):
    col = col.str.lower()
    col = col.str.replace('[^\w\s]', '', regex=True)
    col = col.str.replace('\n', '')
    col = col.str.replace('\d+', '', regex=True)
    col = col.str.replace('\r', '', regex=True)

    return col

In [13]:
df['text'] = nlp_prep(df['text'])

In [14]:
df.head()

,class,text
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...


In [15]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def wc(data, bgcolor='white'):
    plt.figure(figsize=(10, 10))
    mask = np.array(Image.open('heart.png'))
    wc = WordCloud(background_color=bgcolor, stopwords=STOPWORDS, mask=mask)
    wc.generate(' '.join(data))
    plt.imshow(wc)
    plt.axis('off')

In [17]:
ham = df[df['class'] == 0]['text']
spam = df[df['class'] == 1]['text']

In [20]:
wc(ham)

In [21]:
wc(spam)

In [46]:
x = df['text']
y = df['class']

In [47]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [48]:
from sklearn.feature_extraction.text import CountVectorizer as cv
from nltk.stem import PorterStemmer
from textblob import TextBlob

In [49]:
pr = PorterStemmer()

In [50]:
def lemmafn(text):
    words = TextBlob(text).words
    return [pr.stem(word) for word in words]

In [51]:
v = cv(stop_words='english', ngram_range=(1, 2), analyzer=lemmafn)

In [52]:
x = v.fit_transform(x)

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=.2)

In [54]:
mnb = MultinomialNB()

In [55]:
model = mnb.fit(x_train, y_train)

In [56]:
pred = model.predict(x_test)

In [57]:
accuracy_score(pred, y_test)

0.9766816143497757